In [1]:
!pip install fuzzywuzzy
!pip install python-Levenshtein

In [2]:
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [3]:
places = pd.read_csv('locales_202012.csv')
places.drop('Unnamed: 0', axis=1, inplace=True)
terrazas = pd.read_csv('terrazas_limpias.csv')
terrazas.drop('Unnamed: 0', axis=1, inplace=True)

In [4]:
places.drop_duplicates(subset='id_local', inplace=True)

In [5]:
places['rotulo']

0                                            HOTEL MEDIODIA
5                                       LA ESQUINA DE SANTI
6                                            LA DESCUBIERTA
7                                              LA REVOLTOSA
8                                             BARRIGA LLENA
                                ...                        
128915                                   LA CAJA DE SOCORRO
128917                 AHORRAMÁS MERCAMADRID [C1A][EJE 2/7]
128918    SOCIEDAD ESTATAL DE MERCADOS CENTRALES DE ABAS...
128919            EL ENCINAR DE HUMIENTA [P. BAJA] [J1][A] 
128920                                ASEPEYO [J4] [ZONA B]
Name: rotulo, Length: 69694, dtype: object

In [6]:
values_to_drop = ['RÓTULO NO INFORMADO', 'ROTULO NO INFORMADO', 'FARMACIA', 'SR', 'ALIMENTACION','SIN ACTIVIDAD','PELUQUERIA',
                 'CAJA MADRID','SIN ROTULO','MUESTRA','LOCUTORIO','ALIMENTACION FRUTOS SECOS','ALIMENTACIÓN']

In [7]:
places = places[~places['rotulo'].isin(values_to_drop)]

In [8]:
places = places.merge(terrazas['Cod_Postal'], how='left', left_on=places['desc_barrio_local'], right_on=terrazas['desc_barrio_local'])

In [9]:
places.drop_duplicates(subset='id_local', inplace=True)

In [10]:
places.reset_index(drop=True, inplace=True)

In [11]:
places.dropna(how='any', inplace=True)

In [12]:
places['Cod_Postal'] = places['Cod_Postal'].astype(int)

In [13]:
places.reset_index(drop=True, inplace=True)

In [14]:
places_df = places[['rotulo','Cod_Postal']]
terraces_df = terrazas[['rotulo','Cod_Postal']]

In [15]:
list_df= [places_df, terraces_df]

In [16]:
all_places = pd.concat(list_df)

In [17]:
def normalize(s):
    replacements = (
        ("á", "a"),
        ("é", "e"),
        ("í", "i"),
        ("ó", "o"),
        ("ú", "u"),
    )
    for a, b in replacements:
        s = s.replace(a, b).replace(a.upper(), b.upper())
    return s

In [18]:
all_places['rotulo'] = all_places['rotulo'].apply(normalize)

In [19]:
def clean_text(text):
    # Convert text to lowercase
    text = text.lower()
    # Remove punctuation from the text
    text = text.translate(str.maketrans("", "", string.punctuation))
    return text

In [20]:
import string
all_places['rotulo'] = all_places['rotulo'].apply(clean_text)

In [21]:
all_places['rotulo'] = all_places['rotulo'].str.strip()

In [22]:
import re
def remove_duplicates(text):
    regex = r"\b(\w+)\b\s+\b\1\b"
    replacement = r"\1"
    return re.sub(regex, replacement, text)

In [23]:
all_places['name'] = all_places['rotulo'].apply(lambda x: remove_duplicates(x))

In [24]:
all_places['name']

0            hotel mediodia
1       la esquina de santi
2            la descubierta
3              la revoltosa
4             barriga llena
               ...         
6111            bar central
6112                lor ana
6113       bar el acueducto
6114            estacion 55
6115        pizzeria napoli
Name: name, Length: 67887, dtype: object

In [25]:
# group dataframes by zip code and create a list of places for each zip code
places_grouped = all_places.groupby('Cod_Postal')['name'].apply(list).reset_index(name='places')

In [26]:
addresses = pd.read_csv('addresses.csv')
addresses.drop('Unnamed: 0', axis=1, inplace=True)

In [27]:
addresses['result.name'] = addresses['result.name'].apply(normalize)
addresses['result.name'] = addresses['result.name'].apply(clean_text)
addresses['result.name'] = addresses['result.name'].apply(lambda x: remove_duplicates(x))
addresses['result.name'] = addresses['result.name'].str.strip()

In [28]:
addresses

,result.formatted_address,result.formatted_phone_number,result.name,result.place_id,zip_code
0,"C. San Juan de Ortega, 10, 28050 Madrid, Spain",913 07 52 88,restaurante la dehesa,ChIJe8y4CKAuQg0RC4YervOOVLg,28050
1,"Paseo de la, Zona, P.º de la Tierra de Melide,...",917 50 53 93,casa corrochano,ChIJjWIe0PMrQg0RXJxYC8mlRcM,28050
2,"Pl. de la Moraleja, 28109 Alcobendas, Madrid, ...",916 25 25 00,restaurante aspen,ChIJa64cyBIsQg0RkVa57dm7oeY,28109
3,"Pl. de la Moraleja, 1, 28109 Alcobendas, Madri...",916 58 52 97,la maquina la moraleja,ChIJwyguthIsQg0RKfP-___CUFI,28109
4,"C. de Frómista, 14, 28050 Madrid, Spain",914 27 09 45,la mano de pablo,ChIJmw6_CKAuQg0RY9Kcr3tR5OQ,28050
...,...,...,...,...,...
5657,"Av. de la Virgen del Carmen, 55, 28033 Madrid,...",643 38 00 02,istambul doner kebab pizza hortaleza,ChIJlSxL-z8vQg0RXDp-Vq2xFdk,28033
5658,"P.º de la Ermita del Santo, 48, 28011 Madrid, ...",Unknown,restaurante bar,ChIJg6oTi2UnQg0R8d9-s3084EY,28011
5659,"C. de Sepúlveda, 3, 28011 Madrid, Spain",663 51 23 67,restaurante tuc,ChIJVdp3PaMnQg0RX-X78159I14,28011
5660,"P.º de la Ermita del Santo, 48, 28011 Madrid, ...",Unknown,snack bistro,ChIJ59rdMucnQg0RTHjyULEIgus,28011


In [29]:
restaurants_grouped = addresses.groupby('zip_code')['result.name'].apply(list).reset_index(name='google_places')

In [30]:
from fuzzywuzzy import fuzz
from joblib import Parallel, delayed

# define a function to perform fuzzy matching
def fuzzy_match(zip_code, places1, places2, threshold):
    matches = []
    for place1 in places1:
        for place2 in places2:
            if fuzz.token_sort_ratio(place1, place2) >= threshold:
                matches.append((place1, place2))
    return (zip_code, matches)

In [37]:
# set the fuzzy matching threshold to 85 for more accuracy
threshold = 85

# create a list of zip codes to process
zip_codes = restaurants_grouped['zip_code'].unique()

# perform fuzzy matching for each zip code in parallel
results = Parallel(n_jobs=-1)(delayed(fuzzy_match)(
    zip_code,
    places_grouped.loc[places_grouped['Cod_Postal'] == zip_code, 'places'].iloc[0],
    restaurants_grouped.loc[restaurants_grouped['zip_code'] == zip_code, 'google_places'].iloc[0],
    threshold
) for zip_code in zip_codes)

exception calling callback for <Future at 0x1910dd0dd00 state=finished returned list>
Traceback (most recent call last):
  File "C:\Users\Usuario\anaconda3\lib\site-packages\joblib\parallel.py", line 862, in dispatch_one_batch
    tasks = self._ready_batches.get(block=False)
  File "C:\Users\Usuario\anaconda3\lib\queue.py", line 168, in get
    raise Empty
_queue.Empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "C:\Users\Usuario\anaconda3\lib\site-packages\joblib\externals\loky\_base.py", line 26, in _invoke_callbacks
    callback(self)
  File "C:\Users\Usuario\anaconda3\lib\site-packages\joblib\parallel.py", line 385, in __call__
    self.parallel.dispatch_next()
  File "C:\Users\Usuario\anaconda3\lib\site-packages\joblib\parallel.py", line 834, in dispatch_next
    if not self.dispatch_one_batch(self._original_iterator):
  File "C:\Users\Usuario\anaconda3\lib\site-packages\joblib\parallel.py", line 873, in dispatch_

In [38]:
# create a dataframe with the results
df_matches = pd.DataFrame({'zip_code': [r[0] for r in results], 'matches': [r[1] for r in results]})

In [39]:
matches_exploded = df_matches.explode('matches')

In [40]:
matches_exploded.dropna(how='any', inplace=True)
matches_exploded.shape

(2320, 2)

In [41]:
matches_exploded['perfect_match'] = matches_exploded['matches'].apply(lambda x: x[1])

In [42]:
matches_exploded

,zip_code,matches,perfect_match
0,28001,"(restaurante albora, restaurante abbas)",restaurante abbas
0,28001,"(amazonico, amazonico)",amazonico
0,28001,"(wagaboo, wogaboo)",wogaboo
0,28001,"(la maruca, la maruca)",la maruca
0,28001,"(taberna embroque, taberna embroque)",taberna embroque
...,...,...,...
51,28053,"(bar dos angeles, bar dos angeles)",bar dos angeles
51,28053,"(bar restaurante, restaurante bahia)",restaurante bahia
51,28053,"(bar restaurante, restaurante o faro)",restaurante o faro
51,28053,"(bar restaurante, restaurante bahia)",restaurante bahia


In [43]:
matches_str = [str(match) for match in matches_exploded['perfect_match']]

In [44]:
matches_str

['restaurante abbas',
 'amazonico',
 'wogaboo',
 'la maruca',
 'taberna embroque',
 'restaurante abbas',
 'carmen restaurante',
 'restaurante la paloma',
 'restaurante abbas',
 'carmen restaurante',
 'restaurante abbas',
 'carmen restaurante',
 'mercado de la paz',
 'manolita chen',
 'el paraguas',
 'restaurante abbas',
 'carmen restaurante',
 'la casa del abuelo goya',
 'alfredos barbacoa',
 'new york burger',
 'vinotinto',
 'le pain quotidien',
 'arugula',
 'il tavolo verde',
 'wogaboo',
 'el paraguas',
 'la casa del abuelo goya',
 'le pain quotidien',
 'restaurante abbas',
 'vinotinto',
 'restaurante abbas',
 'carmen restaurante',
 'restaurante la paloma',
 'arugula',
 'taberna de la daniela',
 'la trainera',
 'abuelita alicia',
 'parrilla de juan adan',
 'casa vicenta',
 'orale compadre',
 'la taberna del cardenal',
 'fosters hollywood',
 'the market madrid',
 'la vieja alqueria',
 'abuelita ana',
 'el jamon de oro',
 'restaurante lobbo',
 'granier',
 'cafeteria fanny',
 'restauran

In [45]:
addresses['success'] = addresses['result.name'].apply(lambda x: 1 if any(match_str in x for match_str in matches_str) else 0)

In [46]:
google = pd.read_csv('google_final.csv')
google.drop('Unnamed: 0', axis=1,inplace=True)
google.head()

,result.business_status,result.delivery,result.dine_in,result.name,result.place_id,result.rating,result.reservable,result.serves_beer,result.serves_breakfast,result.serves_brunch,...,bar,store,cafe,meal_takeaway,meal_delivery,bakery,liquor_store,night_club,zip_code,price_level
0,OPERATIONAL,True,True,Restaurante La Dehesa,ChIJe8y4CKAuQg0RC4YervOOVLg,4.0,True,True,True,True,...,False,False,False,False,False,False,False,False,28050,2.0
1,OPERATIONAL,True,True,Casa Corrochano,ChIJjWIe0PMrQg0RXJxYC8mlRcM,4.1,True,True,True,True,...,False,False,False,False,False,False,False,False,28050,1.0
2,OPERATIONAL,False,True,Restaurante Aspen,ChIJa64cyBIsQg0RkVa57dm7oeY,4.2,True,True,True,True,...,False,False,False,False,False,False,False,False,28109,4.0
3,OPERATIONAL,False,True,La Máquina La Moraleja,ChIJwyguthIsQg0RKfP-___CUFI,4.2,True,True,False,True,...,False,False,False,False,False,False,False,False,28109,3.0
4,OPERATIONAL,Unknown,True,La Mano de Pablo,ChIJmw6_CKAuQg0RY9Kcr3tR5OQ,4.4,True,True,False,True,...,False,False,False,False,False,False,False,False,28050,2.0


In [47]:
google['result.name'] = google['result.name'].str.strip()
google['result.name'] = google['result.name'].apply(clean_text)
google['result.name'] = google['result.name'].apply(lambda x: remove_duplicates(x))
google['result.name'] = google['result.name'].apply(normalize)

In [48]:
google['success'] = google['result.name'].apply(lambda x: 1 if any(match_str in x for match_str in matches_str) else 0)

In [49]:
google[(google['result.business_status']=='CLOSED_TEMPORARILY')&
      (google['success']==1)]

,result.business_status,result.delivery,result.dine_in,result.name,result.place_id,result.rating,result.reservable,result.serves_beer,result.serves_breakfast,result.serves_brunch,...,store,cafe,meal_takeaway,meal_delivery,bakery,liquor_store,night_club,zip_code,price_level,success
8,CLOSED_TEMPORARILY,Unknown,True,restaurante,ChIJgbXCHjssQg0RzdPHhjiDBlI,3.6,Unknown,True,True,True,...,False,False,False,False,False,False,False,28109,1.0,1
65,CLOSED_TEMPORARILY,True,True,restaurante el pato mudo,ChIJiUlrlSouQg0RnviTSODhHio,3.7,True,True,True,True,...,False,False,False,False,False,False,False,28042,1.0,1
144,CLOSED_TEMPORARILY,True,True,bar asturias restaurante,ChIJ6-URatsuQg0R6k1pCWBXs8Q,4.2,True,True,True,True,...,False,True,False,False,False,False,False,28043,1.0,1
237,CLOSED_TEMPORARILY,Unknown,True,pedro alonso restaurante,ChIJc3Xj5dYuQg0Rg7P339kR5i4,4.2,Unknown,True,True,True,...,False,False,False,False,False,False,False,28043,1.0,1
302,CLOSED_TEMPORARILY,Unknown,True,restaurante doner kebap,ChIJp_DNPrAvQg0RZh9WFIfjJWs,4.3,Unknown,True,True,True,...,False,False,False,False,False,False,False,28022,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5608,CLOSED_TEMPORARILY,True,True,tribar,ChIJUZlMK8kpQg0RisXc6hmt1hQ,4.9,True,True,True,True,...,False,False,False,False,False,False,False,28039,1.0,1
5617,CLOSED_TEMPORARILY,Unknown,True,bar cafe martin,ChIJc3LbY-UuQg0Rukw8cx7u208,3.6,Unknown,True,True,True,...,False,False,False,False,False,False,False,28043,1.0,1
5624,CLOSED_TEMPORARILY,Unknown,True,restaurante,ChIJHwYnNeUuQg0RXQdcgO_rj_M,2.5,Unknown,True,True,True,...,False,False,False,False,False,False,False,28043,1.0,1
5628,CLOSED_TEMPORARILY,Unknown,True,leyali pubrestaurante,ChIJayud6iwmQg0RG4p3lcIjpVE,4.2,Unknown,True,True,True,...,False,False,False,False,False,False,False,28012,1.0,1


In [50]:
#Some restaurants that have closed temporarily seem to be closed definitely, so I will count them as unsuccessful

In [51]:
google.loc[(google['result.business_status'] == 'CLOSED_TEMPORARILY') | (google['result.business_status'] == 'CLOSED_PERMANENTLY'), 'success'] = 0

In [52]:
google[(google['result.business_status']=='CLOSED_TEMPORARILY')&
      (google['success']==1)]

,result.business_status,result.delivery,result.dine_in,result.name,result.place_id,result.rating,result.reservable,result.serves_beer,result.serves_breakfast,result.serves_brunch,...,store,cafe,meal_takeaway,meal_delivery,bakery,liquor_store,night_club,zip_code,price_level,success


In [53]:
google.to_csv('google_target.csv')

In [54]:
google.shape

(5662, 33)